This Python notebook takes a subset of POIs in Downtown Santa Cruz, and moves each one to the closest building.

In [ ]:
!pip install pandas --quiet
!pip install geopandas --quiet
!pip install shapely --quiet
!pip install duckdb --quiet
!pip install jupysql --quiet
!pip install duckdb-engine --quiet
!pip install folium matplotlib mapclassify --quiet

In [ ]:
import pandas as pd
import geopandas as gpd
import duckdb
from shapely import wkt
from shapely import Point
from shapely.ops import nearest_points
import shapely

In [ ]:
%reload_ext sql

In [ ]:
%sql duckdb://

In [ ]:
%sql INSTALL spatial;
%sql INSTALL httpfs;
%sql LOAD spatial;
%sql LOAD httpfs;
%sql SET s3_region='us-west-2'

In [ ]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [ ]:
places = gpd.read_parquet(path='santa_cruz_places_building_snap.parquet')

In [ ]:
gers_ids_list = str(list(places['id'])).replace('[', '(').replace(']', ')')

In [ ]:
%%sql
COPY(
    SELECT
        id,
        tags->>'name' AS name,
        lat,
        lon
    FROM st_readosm('norcal-latest.osm.pbf')
    WHERE lon BETWEEN -122.09 AND -121.91 AND
          lat BETWEEN 36.93 AND 37.02
) TO 'osm_nodes.parquet'

In [ ]:
%%sql
COPY(
WITH ways AS (
    SELECT
        id,
        kind,
        tags ->> 'name' AS name,
        refs
    FROM st_readosm('norcal-latest.osm.pbf')
    WHERE kind = 'way'
      AND name IS NOT NULL
      AND array_length(refs) > 0
),
nodes AS (
    SELECT
        id
    FROM 'osm_nodes.parquet'
)
SELECT
    ways.name,
    ways.refs
FROM ways
JOIN nodes ON nodes.id = ways.refs[1]
) TO 'osm_ways.parquet'

In [ ]:
ways = pd.read_parquet(path='osm_ways.parquet')
ways

In [ ]:
nodes = pd.read_parquet(path='osm_nodes.parquet')
nodes

In [ ]:
ways['lat'] = ways.refs.apply(lambda x: [(nodes[nodes['id'] == v]['lat'].values[0], nodes[nodes['id'] == v]['lon'].values[0]) if len(nodes[nodes['id'] == v]['lat'].values) > 0 else None for v in x])

In [ ]:
ways['lat'] = ways.lat.apply(lambda x: None if None in x else x)

In [ ]:
ways = ways.dropna()
ways

In [ ]:
ways['average'] = ways.lat.apply(lambda x: shapely.centroid(shapely.Polygon(x + [x[0]])))

In [ ]:
ways['lon'] = ways.average.apply(lambda x: x.y)

In [ ]:
ways['lat'] = ways.average.apply(lambda y: y.x)

In [ ]:
ways = ways.drop(['refs', 'average'], axis=1)

In [ ]:
ways

In [ ]:
nodes = nodes.drop('id', axis = 1)

In [ ]:
nodes = nodes.dropna()

In [ ]:
combined = pd.concat([ways, nodes], axis=0)

In [ ]:
combined.to_parquet(path='combined.parquet')

In [ ]:
query_string = f"""
COPY (
    WITH osm AS (
        SELECT
            name,
            lat,
            lon
        FROM 'combined.parquet'
    ),
    overture AS (
        SELECT
            id,
            names.primary AS name,
            geometry AS geometry
        FROM read_parquet('s3://overturemaps-us-west-2/release/2025-04-23.0/theme=places/*/*')
        WHERE id IN {gers_ids_list}
    )
    SELECT
        overture.id AS overture_id,
        overture.name AS overture_name,
        osm.name AS osm_name,
        osm.lon AS lon,
        osm.lat AS lat
    FROM overture
        LEFT JOIN osm ON (
            osm.name = overture.name
            OR osm.name ilike concat('%', overture.name, '%')
            OR overture.name ilike concat('%', osm.name, '%')
            OR damerau_levenshtein(osm.name, overture.name) < 3
        )
        AND st_intersects(
            st_buffer(overture.geometry::geometry, 0.003),
            st_point(osm.lon, osm.lat)
        )
) TO 'ground_truth.parquet'
"""

%sql {{query_string}}

In [ ]:
ground_truth = pd.read_parquet(path='ground_truth.parquet')

In [ ]:
ground_truth[ground_truth['osm_name'] == 'Safeway']

In [ ]:
ground_truth[ground_truth['osm_name'] == 'Pizza My Heart']

In [ ]:
ground_truth.groupby('id').mean().reset_index()

In [ ]:
# Convert places data to GeoFrame for getting places in Point format
places = gpd.GeoDataFrame(
    places,
    geometry=places['geometry'].apply(wkt.loads),
    crs="EPSG:4326"
)